In [1]:
import pandas as pd
import csv

walmart_clean = pd.read_csv("Walmart.csv")
walmart_clean.head()

,Unnamed: 0,Unnamed: 0.1,Store,Date,IsHoliday,Dept,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,...,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Year,Month,Day,Holiday Name
0,0,1,1,2010-02-05,False,1.0,24924.50,42.31,2.572,NaN,...,NaN,NaN,211.096358,8.106,A,151315,2010,2,5,No Holiday
1,1,2,1,2010-02-05,False,26.0,11737.12,42.31,2.572,NaN,...,NaN,NaN,211.096358,8.106,A,151315,2010,2,5,No Holiday
2,2,3,1,2010-02-05,False,17.0,13223.76,42.31,2.572,NaN,...,NaN,NaN,211.096358,8.106,A,151315,2010,2,5,No Holiday
3,3,4,1,2010-02-05,False,45.0,37.44,42.31,2.572,NaN,...,NaN,NaN,211.096358,8.106,A,151315,2010,2,5,No Holiday
4,4,5,1,2010-02-05,False,28.0,1085.29,42.31,2.572,NaN,...,NaN,NaN,211.096358,8.106,A,151315,2010,2,5,No Holiday


In [2]:
walmart_clean['Holiday Name'].value_counts()

No Holiday          396542
Independence Day      8840
Thanksgiving          6004
Christmas             5991
New Year              5948
Name: Holiday Name, dtype: int64

In [3]:
walmart_clean.groupby("Holiday Name")['Weekly_Sales'].mean()

Holiday Name
Christmas           14543.387158
Independence Day    16715.385450
New Year            14862.139543
No Holiday          15909.142723
Thanksgiving        22220.944538
Name: Weekly_Sales, dtype: float64

In [4]:
walmart_groupby = walmart_clean.groupby(['Date', 'Store','Holiday Name'])
walmart_holiday = pd.DataFrame(walmart_groupby.agg({'Weekly_Sales': 'sum', 
                                                    'Temperature': 'mean',
                                                   'CPI': 'mean',
                                                   'Unemployment': 'mean'}).reset_index())
walmart_holiday['Holiday Name'].value_counts()

No Holiday          7605
Independence Day     180
Christmas            135
Thanksgiving         135
New Year             135
Name: Holiday Name, dtype: int64